In [ ]:
!nvidia-smi

!pip install featurewiz==0.2.0

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

archivos = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        archivos.append(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install mlflow


In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools
import seaborn as sns
import category_encoders as ce
import os


from sklearn.tree import plot_tree
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import f1_score, make_scorer, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion    
from sklearn.utils.validation import check_is_fitted

from featurewiz import featurewiz
from scipy import sparse


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

In [7]:
dataset_file = '/kaggle/input/lab1-complete-parquet/downsampled_df.csv'

dataset = pd.read_csv(dataset_file)
dataset.drop(columns='Unnamed: 0', inplace=True)
print(dataset_file)
dataset['clase_ternaria'].replace({'BAJA+2': 1, 'BAJA+1':1, 'CONTINUA':0}, inplace=True)

/kaggle/input/lab1-complete-parquet/downsampled_df.csv


# EDA

In [8]:
def find_cols_with_high_nan(df, threshold=0.80, drop=True): 
    nan_pct = df.isna().sum() / len(df)    
    # Select columns with NaN percentage greater than the threshold
    high_nan_cols = nan_pct[nan_pct > threshold].index.tolist()
    if drop:
        print(f'dropeando {high_nan_cols} porque tienen mas de {threshold} Nan...')
        return df.drop(columns=high_nan_cols)
    return high_nan_cols

def join_visa_master(df):
    print('juntando columnas con prefijo Visa_ y Master_')
    visa_columns = df.filter(like='Visa_').columns.to_list()
    master_columns = df.filter(like='Master_').columns.to_list()
    
    for col_visa, col_master in zip(visa_columns, master_columns):
        new_column_name = 'sum_' + col_visa.split('_', 1)[1]
        
        # Replace NaN values with 0 temporarily for the sum
        visa_values = df[col_visa].fillna(0)
        master_values = df[col_master].fillna(0)
        
        # Sum the non-NaN values and store the result in the new column
        df[new_column_name] = visa_values + master_values
        df[new_column_name] = df[new_column_name].replace(0, np.nan)
    
    return df.drop(columns=visa_columns+master_columns)


def join_cols(df, string):
    joined_cols = df.filter(like=string)
    print(f'sumando las columnas: {joined_cols.columns.to_list()}...')
    
    # Replace NaN values with 0 temporarily for the sum
    joined_cols_filled = joined_cols.fillna(0)
    
    # Calculate the sum
    sum_values = joined_cols_filled.sum(axis=1)
    
    # Replace 0 values with NaN after the summation, but keep the original 0 values when both columns have 0 values in the same row
    df[string] = sum_values.where((joined_cols_filled != 0).any(axis=1), joined_cols_filled.sum(axis=1))
    
    return df.drop(columns=joined_cols.columns)

def find_good_ratios(data):
    '''crea ratios de todas las features float y se queda con los que featurewiz le da importancia, devuelve una lista de buenos candidatos para ratio'''
    def _create_ratios(dataframe, variable,target):
        float_cols = dataframe.select_dtypes(include='float64').columns.tolist()    
        new_dataframe = pd.DataFrame()    
        for col in float_cols:
            if col != variable:
                new_col_name = f'{variable}_to_{col}'
                new_dataframe[new_col_name] = dataframe[variable] / dataframe[col]
                new_dataframe.replace([np.inf, -np.inf], np.nan, inplace=True)

        new_dataframe[target] = dataframe[target]
        return new_dataframe    
    
    the_best = []
    for i in data.select_dtypes(include='float64').columns.tolist():
        temp_df = _create_ratios(data, i,'clase_ternaria')
        try:
            best_ratios = featurewiz(temp_df, 'clase_ternaria',verbose=0)
            the_best.append(best_ratios[0])
        except:
            the_best.append(f'error with {i}')
    return the_best

def create_ratios_from_best(df, lista_best):
    '''crea las features a partir de una lista de listas que tiene los mejores ratios'''
    data = df.copy()    
    for ratios in lista_best:
        if "error" in ratios:
            print(f"Skipping invalid ratio: {ratios}")
            continue
        else:
            for j in ratios:
                variables = j.split('_to_')
                data[j] = data[variables[0]] / data[variables[1]]           
    return data

def preprocessing_dataset(dataset, palabras_agrupar):
    print(dataset.shape)
    dataset = find_cols_with_high_nan(dataset)
    print(dataset.shape)
    dataset = join_visa_master(dataset)
    print(dataset.shape)

    for i in palabras_agrupar:    
        dataset = join_cols(dataset, i)
        print(dataset.shape)
    return dataset

In [9]:
palabras_agrupar =['prestamo','seguro','servicios', 'comisiones','cheques','ahorro','inversion','_consumo','descuentos','tarjeta','consumo','margen','debit','forex', 'transfer','autoservicio','cajas','atm','mobile']

In [11]:
dataset = preprocessing_dataset(dataset,palabras_agrupar)
# list_of_best_ratios = find_good_ratios(dataset)
# dataset = create_ratios_from_best(dataset, list_of_best_ratios)

(86870, 67)
dropeando ['sum_delinquency', 'sum_status', 'sum_msaldodolares', 'sum_madelantopesos', 'sum_madelantodolares', 'sum_mpagado', 'sum_mpagosdolares', 'sum_cadelantosefectivo'] porque tienen mas de 0.8 Nan...
(86870, 59)
juntando columnas con prefijo Visa_ y Master_
(86870, 59)
sumando las columnas: ['prestamo']...
(86870, 58)
sumando las columnas: ['seguro']...
(86870, 57)
sumando las columnas: ['servicios']...
(86870, 56)
sumando las columnas: ['comisiones']...
(86870, 55)
sumando las columnas: ['cheques']...
(86870, 54)
sumando las columnas: ['ahorro']...
(86870, 53)
sumando las columnas: ['inversion']...
(86870, 52)
sumando las columnas: []...
(86870, 53)
sumando las columnas: ['descuentos']...
(86870, 52)
sumando las columnas: ['tarjeta']...
(86870, 51)
sumando las columnas: ['consumo', '_consumo']...
(86870, 49)
sumando las columnas: ['margen']...
(86870, 48)
sumando las columnas: ['debit']...
(86870, 47)
sumando las columnas: ['forex']...
(86870, 46)
sumando las columnas

In [12]:
cols_keep_as_is = ['active_quarter','cliente_vip','internet','tcuentas','cdescubierto_preacordado','ccaja_seguridad','tcallcenter','thomebanking','cplazo_fijo','cajas','mobile','ccuenta_corriente','cpayroll2_trx','ctrx_quarter','cpayroll_trx']
cols_to_binarize = ['cliente_edad','cliente_antiguedad','cproductos','cpagomiscuentas','ccallcenter_transacciones','chomebanking_transacciones','seguro']
cols_to_normalize = [col for col in dataset.select_dtypes('float').columns if col not in cols_keep_as_is + cols_to_binarize]

#len(cols_keep_as_is + cols_to_binarize+cols_to_normalize) == len(set(cols_keep_as_is + cols_to_binarize+cols_to_normalize))
set(dataset.columns) - set(cols_keep_as_is + cols_to_binarize+cols_to_normalize)

{'clase_ternaria', 'foto_mes', 'numero_de_cliente'}

In [46]:
set(dataset.columns) - set(cols_keep_as_is + cols_to_binarize+cols_to_normalize)

set()

In [47]:
# simple type split

cols_to_ignore = ['clase_ternaria', 'foto_mes', 'numero_de_cliente']


cols_to_normalize = dataset.drop(columns=cols_to_ignore).select_dtypes('float').columns.to_list()
cols_keep_as_is = dataset.drop(columns=cols_to_ignore).select_dtypes('int').columns.to_list()


# pipeline preprocessing

In [48]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.columns]


class StandardScalerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        self.scaler = StandardScaler()
        
    def fit(self, X, y=None):
        cols_to_scale = self.columns or X.select_dtypes(include='number').columns
        self.scaler.fit(X[cols_to_scale])
        return self
        
    def transform(self, X):
        cols_to_scale = self.columns or X.select_dtypes(include='number').columns
        X_transformed = pd.DataFrame(self.scaler.transform(X[cols_to_scale]), columns=cols_to_scale, index=X.index)
        return pd.concat([X.drop(columns=cols_to_scale), X_transformed], axis=1)
    
    
class PandasFeatureUnion(BaseEstimator, TransformerMixin):
    def __init__(self, transformers):
        self.transformers = transformers

    def fit(self, X, y=None):
        for _, transformer in self.transformers:
            transformer.fit(X, y)
        return self

    def transform(self, X, y=None):
        transformed_dfs = []
        for _, transformer in self.transformers:
            transformed_dfs.append(transformer.transform(X))        
        
        concatenated_df = pd.concat(transformed_dfs, axis=1)
        return concatenated_df
    

class BinTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, n_bins, columns=None):
        self.n_bins = n_bins
        self.columns = columns
        
    def fit(self, X, y=None):
        self.quantiles_ = {}
        self.outliers_ = {}
        cols_to_transform = self.columns or X.select_dtypes(include=np.number).columns
        for col in cols_to_transform:          
            q1, q3 = np.percentile(X[col], [25, 75])
            iqr = q3 - q1
            lower_bound = q1 - 1.5 * iqr
            upper_bound = q3 + 1.5 * iqr
            mask = (X[col] < lower_bound) | (X[col] > upper_bound)
            self.outliers_[col] = mask
            col_no_outliers = X.loc[~mask, col]
            self.quantiles_[col] = pd.qcut(col_no_outliers, self.n_bins, labels=False, duplicates='drop')
        return self

    def transform(self, X):
        check_is_fitted(self)
        X_trans = X.copy()
        cols_to_transform = self.columns or X_trans.select_dtypes(include=np.number).columns
        for col in cols_to_transform:         
            X_trans[col] = pd.cut(X_trans[col], bins=self.n_bins, labels=False, duplicates='drop')
            X_trans.loc[self.outliers_[col].reindex(X.index, fill_value=False).values, col] = 'outlier'
            if col in self.quantiles_:
                X_trans.loc[~self.outliers_[col].reindex(X.index, fill_value=False), col] = 'bin_' + self.quantiles_[col].apply(lambda x: str(x)).astype(str)      
    
        return X_trans[cols_to_transform]
    
    
class OneHotEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_cols=None):
        self.categorical_cols = categorical_cols
    
    def fit(self, X, y=None):
        if self.categorical_cols is None:
            self.categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
        
        self.categories_ = [X[col].astype('category').cat.categories.tolist() + ['dummy'] for col in self.categorical_cols]
        self.encoder = OneHotEncoder(categories=self.categories_, handle_unknown='ignore')
        self.encoder.fit(X[self.categorical_cols])
        return self
    
    def transform(self, X, y=None):
        X_transformed = self.encoder.transform(X[self.categorical_cols])
        feature_names = self.encoder.get_feature_names_out(self.categorical_cols)
        X_transformed_df = pd.DataFrame.sparse.from_spmatrix(X_transformed, columns=feature_names)
        X_transformed_df.index = X.index
        return pd.concat([X.drop(columns=self.categorical_cols), X_transformed_df], axis=1)


class TargetEncoderWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, cols=None, drop_invariant=False, return_df=True):
        self.cols = cols
        self.drop_invariant = drop_invariant
        self.return_df = return_df
        self.encoder = ce.TargetEncoder(cols=self.cols, drop_invariant=self.drop_invariant, return_df=self.return_df)

    def fit(self, X, y=None):
        self.encoder.fit(X, y)
        return self

    def transform(self, X, y=None):
        return self.encoder.transform(X, y)    
    

class SimpleImputerWrapper(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='mean'):
        self.strategy = strategy
        self.imputer = SimpleImputer(strategy=strategy)
    
    def fit(self, X, y=None):
        self.imputer.fit(X)
        return self
    
    def transform(self, X):
        X_transformed = self.imputer.transform(X)
        return pd.DataFrame(X_transformed, columns=X.columns, index=X.index)
    
    
class PercentageVariationTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, variable):
        self.variable = variable

    def fit(self, X, y):
        # Combine X and y for easier calculations
        data = pd.concat([X, y], axis=1)

        # Calculate the mean of the variable for different target values
        self.target_means = data.groupby(y.name)[self.variable].mean()
        return self

    def transform(self, X, y=None):
        result = pd.DataFrame()

        # Calculate the percentage variation for each target mean
        for target_value, target_mean in self.target_means.items():
            column_name = f"{self.variable}_pct_variation_{target_value}"
            result[column_name] = (X[self.variable] - target_mean) / target_mean * 100

        return result

## feature engineering

In [49]:
class RatioFeature(BaseEstimator, TransformerMixin):
    def __init__(self, base_var, other_vars):
        self.base_var = base_var
        self.other_vars = other_vars if other_vars is not None else []

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Asegurarse de que el input es un DataFrame de pandas
        if not isinstance(X, pd.DataFrame):
            raise ValueError("El input debe ser un DataFrame de pandas")

        # Verificar que base_var existe en el DataFrame
        if self.base_var not in X.columns:
            raise ValueError(f"La columna '{self.base_var}' no se encuentra en el DataFrame")

        # Crear un DataFrame vacío para almacenar las nuevas features
        new_features = pd.DataFrame()

        # Calcular las nuevas features
        for var in self.other_vars:
            if var in X.columns:
                feature_name = f"{self.base_var}_{var}_ratio"
                new_features[feature_name] = X[self.base_var] / X[var]
            else:
                raise ValueError(f"La columna '{var}' no se encuentra en el DataFrame")
        return new_features
    
    
class JoinColumnsTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, like_strings):
        self.like_strings = like_strings

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Asegurarse de que el input es un DataFrame de pandas
        if not isinstance(X, pd.DataFrame):
            raise ValueError("El input debe ser un DataFrame de pandas")

        # Crear un DataFrame vacío para almacenar las nuevas columnas
        new_cols = pd.DataFrame(index=X.index)

        # Iterar sobre las cadenas especificadas
        for like_string in self.like_strings:
            # Filtrar las columnas que contienen la cadena específica
            joined_cols = X.filter(like=like_string)

            # Crear una nueva columna con la suma de las columnas filtradas
            new_col_name = 'sum_'+like_string
            new_cols[new_col_name] = joined_cols.sum(axis=1)
        return new_cols
    


class StandardScalerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        self.scaler = StandardScaler()
        
    def fit(self, X, y=None):
        cols_to_scale = X.select_dtypes(include='number').columns if self.columns is None else self.columns
        self.scaler.fit(X[cols_to_scale])
        return self
        
    def transform(self, X):
        cols_to_scale = X.select_dtypes(include='number').columns if self.columns is None else self.columns
        X_transformed = pd.DataFrame(self.scaler.transform(X[cols_to_scale]), columns=cols_to_scale, index=X.index)
        return pd.concat([X.drop(columns=cols_to_scale), X_transformed], axis=1)    
    


class ReplaceInfTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, factor = 3):
        self.column_max_values = {}
        self.column_min_values = {}
        self.factor = factor

    def fit(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input should be a pandas DataFrame")

        for column in X.columns:
            max_value = X[column].replace([np.inf, -np.inf], np.nan).max()
            min_value = X[column].replace([np.inf, -np.inf], np.nan).min()

            self.column_max_values[column] = max_value
            self.column_min_values[column] = min_value

        return self

    def transform(self, X, y=None):
        if not isinstance(X, pd.DataFrame):
            raise ValueError("Input should be a pandas DataFrame")

        X_copy = X.copy()

        for column in X_copy.columns:
            if column in self.column_max_values:
                X_copy[column] = X_copy[column].replace(np.inf, self.column_max_values[column] * self.factor)
            if column in self.column_min_values:
                X_copy[column] = X_copy[column].replace(-np.inf, self.column_min_values[column] * self.factor)

        return X_copy

In [55]:
# binning = Pipeline([
#     ('col_select',ColumnSelector(columns=cols_to_binarize)),
#     ('bin_convert',BinTransformer(columns=None, n_bins = 3)),
#     ('onehot', OneHotEncoderTransformer())
#     #('target_encoding', TargetEncoderWrapper()) 
# ])

standarize = Pipeline([
    ('col_select',ColumnSelector(columns=cols_to_normalize)),
    ('scale', StandardScalerTransformer(columns=cols_to_normalize)),
])

feature_engineering = PandasFeatureUnion([
    ('as_is', ColumnSelector(columns=cols_keep_as_is)),
    #('log', standarize)   
])

preprocessing = Pipeline([    
    ('inf', ReplaceInfTransformer()),
    #('fillna', SimpleImputerWrapper(strategy='mean')),
    ('feature_engineering', feature_engineering)
])

In [56]:
X = dataset.drop('clase_ternaria', axis=1)
y = dataset['clase_ternaria']

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

# Fit the preprocessing pipeline on the training set and transform both the training and test sets
X_train_processed = preprocessing.fit_transform(X_train, y_train)
X_test_processed = preprocessing.transform(X_test)

# Check the shapes of the processed data
print(X_train_processed.shape, X_test_processed.shape)

(60809, 19) (26061, 19)


## model evaluation

In [60]:
def get_balanced_classes():
    class_counts = y.value_counts()
    total_samples = class_counts.sum()
    class_frequencies = class_counts / total_samples
    class_weights = 1 / class_frequencies
    return class_weights.to_dict()

class_weights_dict = get_balanced_classes()

In [61]:
import mlflow

In [62]:
mlflow.set_experiment("first_baseline")

2023/05/06 19:36:33 INFO mlflow.tracking.fluent: Experiment with name 'first_baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///kaggle/working/mlruns/1', creation_time=1683401793291, experiment_id='1', last_update_time=1683401793291, lifecycle_stage='active', name='first_baseline', tags={}>

In [ ]:
import optuna
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from lightgbm import early_stopping, log_evaluation

def objective(trial):
    
    kf =  KFold(n_splits=3, shuffle=True)    
    f1_scores = []
    
    for train_index, val_index in kf.split(X_train_processed):
        X_tr, X_val = X_train_processed.iloc[train_index], X_train_processed.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        params = {
#             "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0, log=True),
#             "max_depth": trial.suggest_int("max_depth", 1, 10),
#             "num_leaves": trial.suggest_int("num_leaves", 2, 128),  # Fix the maximum value for num_leaves
            
#             'class_weight': class_weights_dict,  
#             "objective": "multiclass",
#             "metric": "multi_logloss",
#             "num_class": 3,
#             "subsample": trial.suggest_float("subsample", 0.4, 1.0),
#             "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
#             "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
#             "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1.0)
            "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.3),
            "max_depth": -1,
            "num_leaves": trial.suggest_int("num_leaves", 8, 1024), 
            "verbosity": -1,
            "boosting_type": "gbdt",   
            'n_jobs': -1,
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000),        
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 50000),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.01, 1.0),
            "extra_trees": True,
            "seed": 42,  # Replace 42 with the desired seed value
            
#             "boosting_type": "gbdt",
#             "objective": "binary",
#             "metric": "custom",
#             "verbosity": -100,
#             'n_jobs': -1,
#             "boost_from_average": True,
#             "feature_pre_filter": False,
#             "force_row_wise": True,
#             "min_gain_to_split": 0.0,
#             "min_sum_hessian_in_leaf": 0.001,
#             "lambda_l1": 0.0,
#             "lambda_l2": 0.0,
#             "max_bin": 31,
#             "num_iterations": 9999,
#             "bagging_fraction": 1.0,
#             "pos_bagging_fraction": 1.0,
#             "neg_bagging_fraction": 1.0,
#             "is_unbalance": False,
#             "scale_pos_weight": 1.0,
#             "drop_rate": 0.1,
#             "max_drop": 50,
#             "skip_drop": 0.5            

        }

        clf = lgb.LGBMClassifier(device='gpu', **params)
        
        try:
            clf.fit(
                X_tr, y_tr, 
                eval_set=[(X_val, y_val)], 
                callbacks=[early_stopping(50), lgb.log_evaluation(period=50)]
            )    
        
        except lgb.basic.LightGBMError as e:
            print(f"Trial {trial.number} failed with parameters: {params}")
            if len(trial.study.trials) > 1:
                print(f"Current best parameters: {trial.study.best_params}")
            return -float('inf')  # Return very low score to skip this set of hyperparameters

            
        y_pred = clf.predict(X_val)
        f1 = f1_score(y_val, y_pred, average='macro')
        f1_scores.append(f1)

    return sum(f1_scores) / len(f1_scores)

In [ ]:
import optuna
import lightgbm as lgb
import mlflow
import mlflow.lightgbm
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
from lightgbm import early_stopping, log_evaluation

# Configurar el experimento en MLflow
mlflow.set_experiment("Optuna_LightGBM_Kaggle")

def objective(trial):
    kf =  KFold(n_splits=3, shuffle=True)

    f1_scores = []
    for train_index, val_index in kf.split(X_train_processed):
        X_tr, X_val = X_train_processed.iloc[train_index], X_train_processed.iloc[val_index]
        y_tr, y_val = y_train.iloc[train_index], y_train.iloc[val_index]
        
        params = {
            "verbosity": -1,
            "boosting_type": "gbdt",
            'n_jobs': -1,
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000), 
            "learning_rate": trial.suggest_float("learning_rate", 0.02, 0.3),
            "max_depth": -1,
            "num_leaves": trial.suggest_int("num_leaves", 8, 1024), 
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 100, 50000),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.01, 1.0),
            "extra_trees": True,
            "seed": 42
        }

        with mlflow.start_run():
            # Registra los parámetros del modelo
            for param_name, param_value in params.items():
                mlflow.log_param(param_name, param_value)
            
            clf = lgb.LGBMClassifier(device='gpu', **params)

            try:
                clf.fit(
                    X_tr, y_tr,
                    eval_set=[(X_val, y_val)],
                    callbacks=[early_stopping(50), lgb.log_evaluation(period=50)]
                )

                y_pred = clf.predict(X_val)
                f1 = f1_score(y_val, y_pred, average='macro')
                f1_scores.append(f1)

                # Registra la métrica F1
                mlflow.log_metric("f1_score", f1)

                # Registra el modelo LightGBM
                mlflow.lightgbm.log_model(clf, "lightgbm_model")

            except lgb.basic.LightGBMError as e:
                print(f"Trial {trial.number} failed with parameters: {params}")
                if len(trial.study.trials) > 1:
                    print(f"Current best parameters: {trial.study.best_params}")
                return -float('inf')

    return sum(f1_scores) / len(f1_scores)

In [ ]:
def optimize_lgbm_hyperparameters():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100)

    print(f"Best trial: {study.best_trial.params}")
    print(f"Best F1 score: {study.best_value}")
    return study.best_trial.params

best_params = optimize_lgbm_hyperparameters()

2023/05/06 19:40:19 INFO mlflow.tracking.fluent: Experiment with name 'Optuna_LightGBM_Kaggle' does not exist. Creating a new experiment.
[I 2023-05-06 19:40:19,429] A new study created in memory with name: no-name-32153ed2-0a92-4b57-ba7b-ee64f9bf6431


[LightGBM] [Warning] min_data_in_leaf is set=10449, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10449
[LightGBM] [Warning] feature_fraction is set=0.2986554182543892, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2986554182543892
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.484294
[100]	valid_0's binary_logloss: 0.454898
[150]	valid_0's binary_logloss: 0.448335
[200]	valid_0's binary_logloss: 0.4462
[250]	valid_0's binary_logloss: 0.445338
[300]	valid_0's binary_logloss: 0.444075
[350]	valid_0's binary_logloss: 0.443247
[400]	valid_0's binary_logloss: 0.44266
[450]	valid_0's binary_logloss: 0.442417
[500]	valid_0's binary_logloss: 0.442185
[550]	valid_0's binary_logloss: 0.441893
[600]	valid_0's binary_logloss: 0.441573
[650]	valid_0's binary_logloss: 0.441144
[700]	valid_0's binary_logloss: 0.440779
[750]	valid_0's binary_logloss: 0.44052
Early stopping, best iteration is:
[743]	valid_

2023/05/06 19:40:32 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpq_i52880/model, flavor: lightgbm), fall back to return ['lightgbm==3.3.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


[LightGBM] [Warning] min_data_in_leaf is set=10449, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10449
[LightGBM] [Warning] feature_fraction is set=0.2986554182543892, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2986554182543892
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.491644
[100]	valid_0's binary_logloss: 0.460658
[150]	valid_0's binary_logloss: 0.452501
[200]	valid_0's binary_logloss: 0.45046
[250]	valid_0's binary_logloss: 0.450161
[300]	valid_0's binary_logloss: 0.449126
[350]	valid_0's binary_logloss: 0.448566
[400]	valid_0's binary_logloss: 0.447837
[450]	valid_0's binary_logloss: 0.447722
[500]	valid_0's binary_logloss: 0.447626
[550]	valid_0's binary_logloss: 0.447358
[600]	valid_0's binary_logloss: 0.447089
[650]	valid_0's binary_logloss: 0.446846
[700]	valid_0's binary_logloss: 0.446773
[750]	valid_0's binary_logloss: 0.446631
[800]	valid_0's binary_logloss: 0.446494
[85

2023/05/06 19:40:38 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpb9o1_kfd/model, flavor: lightgbm), fall back to return ['lightgbm==3.3.2', 'cloudpickle==2.2.1']. Set logging level to DEBUG to see the full traceback.


...

[LightGBM] [Warning] min_data_in_leaf is set=11469, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=11469
[LightGBM] [Warning] feature_fraction is set=0.48218635106797, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.48218635106797
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.538707
[100]	valid_0's binary_logloss: 0.501199
[150]	valid_0's binary_logloss: 0.486722
[200]	valid_0's binary_logloss: 0.478583
[250]	valid_0's binary_logloss: 0.477662
[300]	valid_0's binary_logloss: 0.472794
[350]	valid_0's binary_logloss: 0.471483
[400]	valid_0's binary_logloss: 0.470311
[450]	valid_0's binary_logloss: 0.469736
[500]	valid_0's binary_logloss: 0.469069
[550]	valid_0's binary_logloss: 0.468943
[600]	valid_0's binary_logloss: 0.468723
[650]	valid_0's binary_logloss: 0.468611
[700]	valid_0's binary_logloss: 0.468526
[750]	valid_0's binary_logloss: 0.468409
[800]	valid_0's binary_logloss: 0.468238
Did no

[I 2023-05-06 19:41:22,346] Trial 2 finished with value: 0.7958439392683351 and parameters: {'n_estimators': 832, 'learning_rate': 0.08779099158601494, 'num_leaves': 365, 'min_data_in_leaf': 11469, 'feature_fraction': 0.48218635106797}. Best is trial 0 with value: 0.804196514882748.


[LightGBM] [Warning] min_data_in_leaf is set=34339, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=34339
[LightGBM] [Warning] feature_fraction is set=0.20929673124770984, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.20929673124770984
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.693167
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.693167
[LightGBM] [Warning] min_data_in_leaf is set=34339, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=34339
[LightGBM] [Warning] feature_fraction is set=0.20929673124770984, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.20929673124770984
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.693141
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.693141
[LightGBM] [Warning] min_data_in_leaf is set=34339, min_child_samples=20 will be i

[I 2023-05-06 19:41:35,093] Trial 3 finished with value: 0.33389139517739097 and parameters: {'n_estimators': 972, 'learning_rate': 0.21682736037171904, 'num_leaves': 889, 'min_data_in_leaf': 34339, 'feature_fraction': 0.20929673124770984}. Best is trial 0 with value: 0.804196514882748.


[LightGBM] [Warning] min_data_in_leaf is set=18342, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18342
[LightGBM] [Warning] feature_fraction is set=0.5847561406179358, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5847561406179358
Training until validation scores don't improve for 50 rounds
[50]	valid_0's binary_logloss: 0.620124
[100]	valid_0's binary_logloss: 0.579883
[150]	valid_0's binary_logloss: 0.56197
[200]	valid_0's binary_logloss: 0.548941
[250]	valid_0's binary_logloss: 0.540275
[300]	valid_0's binary_logloss: 0.540188
[350]	valid_0's binary_logloss: 0.5308
[400]	valid_0's binary_logloss: 0.523748
Early stopping, best iteration is:
[373]	valid_0's binary_logloss: 0.523739
[LightGBM] [Warning] min_data_in_leaf is set=18342, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=18342
[LightGBM] [Warning] feature_fraction is set=0.5847561406179358, colsample_bytree=1.0 will be ignored. Current value: feature_fra

In [64]:
mlflow ui

SyntaxError: invalid syntax (3574192917.py, line 1)

In [ ]:
best_params

In [ ]:
import shap
from lightgbm import LGBMClassifier

#class_weight = {0:1, 1:40}

model = LGBMClassifier(device='gpu', **best_params)

model.fit(X_train_processed, y_train)

y_pred_proba = model.predict_proba(X_test_processed)

In [ ]:
y_pred = model.predict(X_test_processed)

cm = confusion_matrix(y_test, y_pred)
ax = sns.heatmap(cm, annot=True, fmt='d', cmap="Blues")
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
plt.show()
print(classification_report(y_test, y_pred))

# prediccion

In [ ]:
# Read the dataset
predict_set = pd.read_csv('/kaggle/input/dset-peq/dataset_pequeno.csv')

dapply = predict_set[predict_set['foto_mes'] == 202109].drop('clase_ternaria', axis=1)

In [ ]:
dapply = preprocessing_dataset(dapply,palabras_agrupar)

In [ ]:
dapply = create_ratios_from_best(dapply, list_of_best_ratios)

In [ ]:
y_pred_result = model.predict_proba(preprocessing.transform(dapply))

# Create a DataFrame for predicted labels using the same index as numero_de_cliente
predicted_df = pd.DataFrame(y_pred_result[:, 1], columns=['proba'], index=dapply.index)

# Combine the numero_de_cliente and predicted labels
end_res = pd.concat([dapply['numero_de_cliente'], predicted_df], axis=1)

In [ ]:
def get_nth(df_end, n):
    end_res = df_end.copy()
    get_nth = end_res.sort_values('proba', ascending=False).head(n).iloc[-1]['proba']
    end_res['Predicted'] = (end_res['proba'] >= get_nth).astype(int)
    end_res.drop(columns='proba', inplace=True)
    return end_res

In [ ]:
end_res

In [ ]:
n = 9000

end_result = get_nth(end_res,n=n)
end_result['Predicted'].value_counts()

In [ ]:
end_result.to_csv("K101_001.csv", index=False)

In [ ]:
import os
os.environ['KAGGLE_USERNAME'] = 'lechuck666'
os.environ['KAGGLE_KEY'] = '3aaf0a00c8b35504b64d43d4592d6caf'
!kaggle competitions submit -c laboratorio-de-imp-i-2023-virtual -f ./K101_001.csv -m "2nd_less var"